<a href="https://colab.research.google.com/github/anastasiagorbunovaso/Neural-networks/blob/master/laba1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Загрузка и нормализация CIFAR10
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.optim as optim
# visualisation
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
#Определение свёрточной нейронной сети
##Копирование нейросети из раздела Neural Networks и изменение её таким образом, 
##чтобы она получала 3-канальные изображения 
##(вместо 1-канальных изображений, как было определено)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
#Определение функции потери и оптимизатор
##Использование Classification Cross-Entropy loss и SGD with momentum.
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
#Тренировка сети
df_results = pd.DataFrame(columns=['epoch', 'train_acc', 'train_loss', 'test_acc', 'test_loss'])
epochs = 50
import time
start_time = time.time()
for epoch in range(epochs):
    print('\n===> epoch %d' % epoch)
    running_loss = 0.0
    df_results.loc[epoch,['epoch']] = epoch+1
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
    
    correct = 0
    total = 0
    with torch.no_grad():
        train_loss = 0.0
        for data in trainloader:
            inputs, labels = data
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()       
            loss = criterion(outputs, labels)
            train_loss += loss.item()  
    train_acc = 100 * correct / total
    df_results.loc[epoch,['train_acc']] = train_acc
    df_results.loc[epoch,['train_loss']] = train_loss/len(trainloader)
    
    
    correct = 0
    total = 0
    with torch.no_grad():
        test_loss = 0.0
        for data in testloader:
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()       
            loss = criterion(outputs, labels)
            test_loss += loss.item() 
    test_acc= 100 * correct / total
    df_results.loc[epoch,['test_acc']] = test_acc
    df_results.loc[epoch,['test_loss']] = test_loss/len(testloader)
    print("-— %s seconds —-" % (time.time() - start_time))
    start_time = time.time()
#Итоговая таблица
df_results
#Построение графиков
##Зависимость функции потерь от номера эпохи для обучающих данных
plt.plot(df_results['epoch'],df_results['train_loss']);
plt.title("Функция потерь от номера эпохи для обучающих данных") 
plt.xlabel("Эпоха") 
plt.ylabel("Функция потерь");
plt.savefig('Зависимость функции потерь от номера эпохи для обучающих данных.jpg')
##Зависимость метрики качества от номера эпохи для обучающих данных
plt.plot(df_results['epoch'],df_results['train_acc']);
plt.title("Метрика качества от номера эпохи для обучающих данных") 
plt.xlabel("Эпоха") 
plt.ylabel("Метрика качества");
plt.savefig('Зависимость метрики качества от номера эпохи для обучающих данных.jpg')
##Зависимость функции потерь от номера эпохи для контрольных данных
plt.plot(df_results['epoch'],df_results['test_loss']);
plt.title("Функция потерь от номера эпохи для контрольных данных") 
plt.xlabel("Эпоха") 
plt.ylabel("Функция потерь");
plt.savefig('Зависимость функции потерь от номера эпохи для контрольных данных.jpg')
##Зависимость метрики качества от номера эпохи для контрольных данных
plt.plot(df_results['epoch'],df_results['test_acc']);
plt.title("Метрика качества от номера эпохи для контрольных данных") 
plt.xlabel("Эпоха") 
plt.ylabel("Метрика качества");
plt.savefig('Зависимость метрики качества от номера эпохи для контрольных данных.jpg')
#Сохранение графиков
from google.colab import files
files.download( "Зависимость функции потерь от номера эпохи для обучающих данных.jpg" )  
files.download( "Зависимость метрики качества от номера эпохи для обучающих данных.jpg" )  
files.download( "Зависимость функции потерь от номера эпохи для контрольных данных.jpg" )  
files.download( "Зависимость метрики качества от номера эпохи для контрольных данных.jpg" )  
#Сохранение обученной модели
PATH = 'C:/Users/Admin/Downloads'
torch.save(net.state_dict(), PATH)